# Denoising Data with ICA

## Aggressive Denoising

````{tab} Python
```python
from nilearn import glm
```
````
````{tab} AFNI
```bash
3dcalc --input stuff
```
````

## Non-Aggressive Denoising


## Component orthogonalization